<a href="https://colab.research.google.com/github/cgihlstorf/AIAgents/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch datasets accelerate tqdm bitsandbytes matplotlib seaborn